<a href="https://colab.research.google.com/github/HoTomy/ChordGen/blob/main/ModelBuild.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Music21**

In [ ]:
pip install music21

**Connect Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Install Pretty MIDI**

In [ ]:
pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.7 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=96924f763e6f8741cc7ce6239ad3eeccb208232503b1544b557b44563d8a82da
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


**Import Suitable Library**

In [ ]:
import pretty_midi
import numpy as np
from music21 import converter

**Install MIDO**

In [ ]:
pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.7 MB/s eta 0:00:00


**Load the MIDI files using the mido library**

In [ ]:
import mido

# Define the path to your MIDI files
# Directory where the MIDI files are located

midi_files_path = "/content/drive/MyDrive/midi"

# Load MIDI files
midi_files = []
for i in range(70):
    filename = f"{midi_files_path}/file{i:02d}.mid"
    midi = mido.MidiFile(filename)
    midi_files.append(midi)

**Extract the chord information using the mido library**

In [ ]:
from mido import MidiFile

def extract_chords(midi):
    chords = []
    current_chord = set()

    for message in midi:
        if message.type == 'note_on':
            if message.velocity > 0:
                current_chord.add(message.note)
            else:
                current_chord.remove(message.note)

            if len(current_chord) > 0:
                chords.append(current_chord.copy())
            else:
                chords.append(set())

    return chords

chords_list = []
for midi in midi_files:
    chords = extract_chords(midi)
    chords_list.append(chords)

**Analyze the chords using the music21 library to identify triads**

In [ ]:
from music21 import chord


def analyze_chords(chords):
    triads = []
    for chord_set in chords:
        triad_set = set()
        for chord_notes in chord_set:
            chord_obj = chord.Chord(chord_notes)
            if chord_obj.isTriad() and chord_obj.isMajorTriad():
                triad_set.add(chord_obj.pitchedCommonName)
        triads.append(triad_set)
    return triads

triads_list = []
for chords in chords_list:
    triads = analyze_chords(chords)
    triads_list.append(triads)

**Preprocess the data**

In [ ]:
def preprocess_data(triads):
    chord_equations = []
    for triad_set in triads:
        equation_set = set()
        for triad in triad_set:
            root, quality = triad.split()
            if quality.upper() in ['MAJOR', 'MINOR']:
                equation_set.add(f"{root.upper()} {quality.upper()}")
        chord_equations.append(equation_set)
    return chord_equations

chord_equations_list = []
for triads in triads_list:
    chord_equations = preprocess_data(triads)
    chord_equations_list.append(chord_equations)

**Train the regression model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assign numerical labels to the chord equations
labels = list(set(flat_chord_equations))
label_to_index = {label: i for i, label in enumerate(labels)}
indexed_labels = [label_to_index[equation] for equation in flat_chord_equations]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(chords_list, indexed_labels, test_size=0.2, random_state=42)

# Train the linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)




**Print out 24 regression equations in matrix**



In [ ]:
# Train the linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Create a mapping from numerical labels to chord equations
index_to_label = {i: label for label, i in label_to_index.items()}

# Convert the predicted numerical labels to chord equations
predicted_equations = [index_to_label[int(round(prediction))] for prediction in y_pred]

# Print the matrix of predicted chord equations
matrix = [[predicted_equations[i + j] for j in range(12)] for i in range(0, 24, 12)]
for row in matrix:
    print(row)